In [ ]:
import numpy as np
import random
import os
import csv
import gurobipy as gp
from gurobipy import GRB

In [ ]:
# Choose model parameters for set of test instances

T = 336 # Number of time periods
k = 3   # Number of rounds played (supports 2-4)
C = 1   # Capacity (max. matches playable per time period)
W = 0.1 # Minimum willingness threshold
ALPHA = 10 # Coefficient in objective weighted on overall willingness


P = 2 ** k # number of players
mpr = 2 ** (k-1) # max matches per round

In [ ]:
# Create (random) test instances of some size
import numpy as np
import random
import os

N = 20  # Choose number of test instances

P = 2 ** k # number of players

a = np.zeros((P, T))
for n in range(N):
    for p in range(P):
        for t in range(T):
            a[p][t] = random.randrange(11) / 10
    testfile = os.getcwd() + r'\tests\a_' + str(k) + "_" + str(T) + "_" + str(n+1) + ".csv"
    if not os.path.exists(os.path.dirname(testfile)):
        os.makedirs(os.path.dirname(testfile))
    np.savetxt(testfile, a, delimiter=',', fmt = "%.1f")

In [ ]:
# Load schedule data of correct size (indicating required availability for each player for each match)
# Already pre-made for 2, 3, 4 rounds in files b_2, b_3, b_4.csv
import csv

mpr = 2 ** (k-1) # max matches per round
b = np.zeros((P, k, mpr)) # not all of third dimension used, some matches will "not exist" and corresponding entries remain 0

b_file = os.getcwd() + r'\schedules\b_' + str(k) + ".csv" # location  + name of schedule file for k rounds
with open(b_file, newline='', encoding="utf-8-sig") as csvfile:
    b_reader = csv.reader(csvfile)
    p = 0
    for row in b_reader:
        i = 0
        j = 0
        col = 0
        while col < (P - 1):   # P - 1 total matches required, all in a row in order of round first, then match within round
            while j < 2 ** (k - (i + 1)): # 2**(k-(i+1)) matches within round i (zero-indexed)
                b[p][i][j] = int(row[col])
                j += 1
                col += 1
            i += 1   # change round number when round over
            j = 0
        p += 1 # change player number when row finished

In [ ]:
# Running models and writing results for all test instances

for n in range(N):
    a = np.zeros((P, T))
    testfile = os.getcwd() + r'\tests\a_' + str(k) + "_" + str(T) + "_" + str(n+1) + ".csv"
    with open(testfile, newline='', encoding="utf-8-sig") as csvfile:
        a_reader = csv.reader(csvfile)
        p = 0
        for row in a_reader:
            for t in range(T):
                a[p][t] = float(row[t])
            p += 1
    M = gp.Model()

    c = np.array([i + 1 for i in range(T)])

    x = M.addMVar(shape = T, vtype = 'B', name = "x")
    y = M.addMVar(shape = T, vtype = 'B', name = "y")
    e = M.addMVar(shape = T, vtype = 'B', name = "e")
    r = M.addMVar(shape = (k, T), vtype = 'B', name = "r")
    m = M.addMVar(shape = (k, mpr, T), vtype = 'B', name = "m")
    z = M.addMVar(shape = (k, mpr, T), vtype = 'B', name = "z")

    M.addConstr(x.sum() == 1)
    M.addConstr(x[0] == y[0]) 
    for t in range(1, T):
        M.addConstr(x[t] <= y[t])
        M.addConstr(x[t] >= y[t] - y[t-1])
        M.addConstr(x[t] <= 1 - y[t-1])

    for t in range(T):
        M.addConstr(gp.quicksum(m[i][j][t] for i in range(k) for j in range(2 ** (k-(i+1)))) <= C) # not exceeding per time-period capacity 

    for i in range(k):
        for j in range(2 ** (k-(i + 1))):
            M.addConstr(m[i][j].sum() == 1) # each match played once
            M.addConstr(m[i][j][0] <= y[0])
            M.addConstr(m[i][j][0] == z[i][j][0]) 
            for t in range(1, T):
                M.addConstr(m[i][j][t] <= y[t])
                M.addConstr(m[i][j][t] <= z[i][j][t])
                M.addConstr(m[i][j][t] >= z[i][j][t] - z[i][j][t-1])
                M.addConstr(m[i][j][t] <= 1 - z[i][j][t-1])
                M.addConstr(r[i][t] <= z[i][j][t]) 
        M.addConstr(r[i][0] == 0)

    for i in range(1, k):
        for j in range(2 ** (k-(i + 1))):
            for t in range(T):
                M.addConstr(m[i][j][t] <= r[i-1][t])

    for p in range(P):
        for i in range(k):
            for j in range(2 ** (k-(i + 1))):
                for t in range(T):
                    M.addConstr(m[i][j][t] - a[p][t] <= 2 - W - b[p][i][j])  


    M.addConstr(e.sum() == 1)
    for t in range(T):
        M.addConstr(e[t] <= r[k-1][t])

    M.setObjective(gp.quicksum(c[t]*(e[t]-x[t]) for t in range(T)) - ALPHA*gp.quicksum(m[i][j][t]*b[p][i][j]*a[p][t]*(1 / (2**(i+1))) for p in range(P) for t in range(T) for i in range(k) for j in range(2 ** (k-(i+1)))), GRB.MINIMIZE)
    M.optimize()
    resultsfile = os.getcwd() + r'\results\w' + str(W) + "_alpha" + str(ALPHA) + r'\m1_' + str(k) + "_" + str(T) + "_" + str(n+1) + ".json"
    if not os.path.exists(os.path.dirname(resultsfile)):
        os.makedirs(os.path.dirname(resultsfile))
    M.write(resultsfile)
    
    M2 = gp.Model()

    x = M2.addMVar(shape = T, vtype = 'B', name = "x")
    y = M2.addMVar(shape = T, vtype = 'B', name = "y")
    e = M2.addMVar(shape = T, vtype = 'B', name = "e")
    f = M2.addMVar(shape = T, vtype = 'B', name = "f")
    r = M2.addMVar(shape = (k, T), vtype = 'B', name = "r")
    m = M2.addMVar(shape = (k, mpr, T), vtype = 'B', name = "m")
    z = M2.addMVar(shape = (k, mpr, T), vtype = 'B', name = "z")

    M2.addConstr(x[0] == y[0])
    M2.addConstr(y[T-1] == 1)

    for t in range(1, T):
        M2.addConstr(y[t] == gp.quicksum(x[q] for q in range(t+1)))
        M2.addConstr(x[t] == y[t] - y[t-1])

    for t in range(T):
        M2.addConstr(gp.quicksum(m[i][j][t] for i in range(k) for j in range(2 ** (k-(i+1)))) <= C)

    for i in range(k):
        for j in range(2 ** (k-(i + 1))):
            M2.addConstr(m[i][j][0] <= y[0])
            M2.addConstr(m[i][j][0] == z[i][j][0])
            M2.addConstr(z[i][j][T-1] == 1)
            for t in range(1, T):
                M2.addConstr(z[i][j][t] == gp.quicksum(m[i][j][q] for q in range(t+1)))
                M2.addConstr(m[i][j][t] <= y[t])
                M2.addConstr(m[i][j][t] == z[i][j][t] - z[i][j][t-1])
                M2.addConstr(r[i][t] <= z[i][j][t])
        M2.addConstr(r[i][0] == 0)

    for p in range(P):
        for i in range(k):
            for j in range(2 ** (k-(i + 1))):
                for t in range(T):
                    M2.addConstr(m[i][j][t] - a[p][t] <= 2 - W - b[p][i][j])
                
    for i in range(1, k):
        for j in range(2 ** (k-(i + 1))):
            for t in range(T):
                M2.addConstr(m[i][j][t] <= r[i-1][t])

    M2.addConstr(e[0] == f[0])
    M2.addConstr(e[0] <= r[k-1][0])
    M2.addConstr(f[T-1] == 1)
    for t in range(1, T):
        M2.addConstr(f[t] == gp.quicksum(e[q] for q in range(t+1)))
        M2.addConstr(e[t] <= r[k-1][t])
        M2.addConstr(e[t] == f[t] - f[t-1])

    M2.setObjective(gp.quicksum(c[t]*(e[t]-x[t]) for t in range(T)) - ALPHA*gp.quicksum(m[i][j][t]*b[p][i][j]*a[p][t]*(1 / (2**(i+1))) for p in range(P) for t in range(T) for i in range(k) for j in range(2 ** (k-(i+1)))), GRB.MINIMIZE)
    M2.optimize()
    M2.write(os.getcwd() + r'\results\w' + str(W) + "_alpha" + str(ALPHA) + r'\m2_' + str(k) + "_" + str(T) + "_" + str(n+1) + ".json")